## 测试spliter功能

In [1]:
# from dataloaders import ImageNetR_Spliter

# spliter = ImageNetR_Spliter(client_num=5, task_num=5, private_class_num=40, input_size=224, path='C:/Users/Admin/datasets')

In [2]:
# client_subset,client_mask = spliter.random_split()

In [3]:
# del client_subset,client_mask,spliter

In [4]:
from dataloaders import Cifar100_Spliter

spliter = Cifar100_Spliter(client_num=5, task_num=5, private_class_num=15, input_size=224, path='C:/Users/Admin/datasets')
client_subset,client_mask = spliter.random_split()

print(client_mask)

# del client_subset,client_mask,spliter

Files already downloaded and verified


100%|██████████| 50000/50000 [00:01<00:00, 39286.54it/s]


[[[52, 15, 44, 83, 73, 33, 10, 78], [65, 42, 22, 38, 98, 25, 32, 57], [59, 66, 13, 40, 61, 77, 5, 63], [43, 94, 90, 46, 55, 6, 26, 79], [39, 7, 34, 53, 27, 16, 29, 47]], [[15, 30, 46, 42, 22, 63, 66, 29], [47, 55, 72, 9, 17, 77, 60, 11], [83, 59, 1, 95, 87, 0, 20, 98], [53, 79, 78, 13, 34, 12, 2, 32], [14, 85, 94, 27, 65, 7, 33, 61]], [[61, 19, 27, 23, 31, 32, 99, 7], [69, 15, 53, 55, 78, 66, 13, 29], [46, 83, 28, 22, 51, 82, 33, 98], [54, 50, 34, 59, 77, 47, 79, 63], [88, 41, 42, 68, 84, 80, 65, 94]], [[74, 32, 33, 35, 15, 78, 97, 65], [98, 77, 92, 3, 46, 29, 18, 83], [27, 59, 13, 91, 7, 61, 93, 67], [70, 79, 47, 36, 66, 53, 55, 24], [63, 89, 42, 94, 34, 22, 4, 45]], [[63, 86, 94, 42, 33, 96, 15, 13], [81, 34, 78, 79, 27, 59, 61, 71], [77, 56, 98, 8, 53, 22, 37, 49], [29, 64, 21, 47, 48, 32, 58, 66], [65, 75, 7, 46, 55, 62, 83, 76]]]


In [5]:
_, test = spliter.process_testdata(0)
print(client_mask)
print(len(test))

100%|██████████| 100/100 [00:00<00:00, 21306.02it/s]


[[[52, 15, 44, 83, 73, 33, 10, 78], [65, 42, 22, 38, 98, 25, 32, 57], [59, 66, 13, 40, 61, 77, 5, 63], [43, 94, 90, 46, 55, 6, 26, 79], [39, 7, 34, 53, 27, 16, 29, 47]], [[15, 30, 46, 42, 22, 63, 66, 29], [47, 55, 72, 9, 17, 77, 60, 11], [83, 59, 1, 95, 87, 0, 20, 98], [53, 79, 78, 13, 34, 12, 2, 32], [14, 85, 94, 27, 65, 7, 33, 61]], [[61, 19, 27, 23, 31, 32, 99, 7], [69, 15, 53, 55, 78, 66, 13, 29], [46, 83, 28, 22, 51, 82, 33, 98], [54, 50, 34, 59, 77, 47, 79, 63], [88, 41, 42, 68, 84, 80, 65, 94]], [[74, 32, 33, 35, 15, 78, 97, 65], [98, 77, 92, 3, 46, 29, 18, 83], [27, 59, 13, 91, 7, 61, 93, 67], [70, 79, 47, 36, 66, 53, 55, 24], [63, 89, 42, 94, 34, 22, 4, 45]], [[63, 86, 94, 42, 33, 96, 15, 13], [81, 34, 78, 79, 27, 59, 61, 71], [77, 56, 98, 8, 53, 22, 37, 49], [29, 64, 21, 47, 48, 32, 58, 66], [65, 75, 7, 46, 55, 62, 83, 76]]]
50000


In [6]:
from trainer import Trainer
from utils.options import get_args
args = get_args("")

c:\Users\Admin\anaconda3\envs\fcl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import os
from utils.utils import Logger
import sys
import yaml
import numpy as np
# duplicate output stream to output file
if not os.path.exists(args.log_dir): os.makedirs(args.log_dir)
log_out = args.log_dir + '/output.log'
sys.stdout = Logger(log_out)

# save args
with open(args.log_dir + '/args.yaml', 'w') as yaml_file:
    yaml.dump(vars(args), yaml_file, default_flow_style=False)

metric_keys = ['acc','time',]
save_keys = ['global', 'pt', 'pt-local']
global_only = ['time']
avg_metrics = {}
for mkey in metric_keys: 
    avg_metrics[mkey] = {}
    for skey in save_keys: avg_metrics[mkey][skey] = []
    
# load results
if args.overwrite:
    start_r = 0
else:
    try:
        for mkey in metric_keys: 
            for skey in save_keys:
                if (not (mkey in global_only)) or (skey == 'global'):
                    save_file = args.log_dir+'/results-'+mkey+'/'+skey+'.yaml'
                    if os.path.exists(save_file):
                        with open(save_file, 'r') as yaml_file:
                            yaml_result = yaml.safe_load(yaml_file)
                            avg_metrics[mkey][skey] = np.asarray(yaml_result['history'])
        # next repeat needed
        start_r = avg_metrics[metric_keys[0]][save_keys[0]].shape[-1]
        # extend if more repeats left
        if start_r < args.repeat:
            max_task = avg_metrics['acc']['global'].shape[0]
            for mkey in metric_keys: 
                avg_metrics[mkey]['global'] = np.append(avg_metrics[mkey]['global'], np.zeros((max_task,args.repeat-start_r)), axis=-1)
                if (not (mkey in global_only)):
                    avg_metrics[mkey]['pt'] = np.append(avg_metrics[mkey]['pt'], np.zeros((max_task,max_task,args.repeat-start_r)), axis=-1)
                    avg_metrics[mkey]['pt-local'] = np.append(avg_metrics[mkey]['pt-local'], np.zeros((max_task,max_task,args.repeat-start_r)), axis=-1)
    except:
        start_r = 0

====================== 1 =======================
Optimizer is reset!
LR: 0.001


In [8]:
from torch.utils.data import random_split

client_data_train = []
client_data_val = []
for subset in client_subset:
	temp_train = []
	temp_val =[]
	for data in subset:
		train_dataset, val_dataset = random_split(data, [int(len(data) * 0.7), len(data) - int(len(data) * 0.7)])
		temp_train.append(train_dataset)
		temp_val.append(val_dataset)
	client_data_train.append(temp_train)
	client_data_val.append(temp_val)
 
surro_data, test_data = spliter.process_testdata(5)
Trainer(args, 42, metric_keys, save_keys, train_dataset=client_data_train[0], test_dataset=test_data, 
        validate_dataset=client_data_val[0], client_index=1, class_mask=client_mask[0]).train(avg_metrics)

100%|██████████| 100/100 [00:00<00:00, 16671.85it/s]
c:\Users\Admin\repo\FCL-Rebuild\learners\default.py:220: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_s

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
